# Workshop 2 - Cheminformatics with RDKit

In this workshop, we'll further explore the chemical dataset we explored last week, using RDKit to do some Cheminformatics and connect chemical properties to physical properties.

## Useful resources

We will be using some of the python libraries you have already seen and RDKit, which you briefly encountered last term. Here are some quick start guides and/or tutorials that might come in useful.

- Pandas
  - [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)
- Seaborn
  - Charles J. Weis [seaborn tutorial](https://weisscharlesj.github.io/SciCompforChemists/notebooks/chapter_10/chap_10_notebook.html)
- RDKit
  - [Getting started with the RDKit in Python](https://www.rdkit.org/docs/GettingStartedInPython.html)
  - [RDKit tutorial from 2021](https://github.com/greglandrum/AIDD_RDKit_Tutorial_2021/blob/b4c4661ff7980721823654f54cd0c28031c5884c/RDKit_Intro.ipynb) - this covers a lot of ground. We won't be talking about reactions (towards end of notebook)
  - There are also lots of videos on YouTube and of course ChatGPT (though I am not sure how well it does with RDKit, probably because the documentation is patchy).
  - Charles J. Weis [RDKit tutorial](https://weisscharlesj.github.io/SciCompforChemists/notebooks/chapter_15/chap_15_notebook.html).
- SMILES Strings
  - [Smiles Strings for Beginners](https://chemicbook.com/2021/02/13/smiles-strings-explained-for-beginners-part-1.html)
  - [An Introduction to SMILES Strings](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html)


You might also find some useful bits and pieces in the [Molecular fingerprints notebook](https://joeforth.github.io/chem502_book/chem-data/descriptors-similarity/) in the module book.


## 0. Importing Libraries

Let's start by importing some libraries:

- time (needed to include a sleep)
- requests
- pandas 
- numpy
- seaborn

In [ ]:
# TODO: Write your import statements.

# rdkit has a complicated structure, so we will start with these and maybe add some later

from rdkit import Chem
from rdkit.Chem import (
                        AllChem,
                        rdCoordGen,
                        Draw,
                        rdFingerprintGenerator,
                        PandasTools,
                        Descriptors,
                        DataStructs
                        )

from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs

from IPython.display import SVG
from ipywidgets import interact,fixed,IntSlider

## 1. Load (and Clean) the Data

Load the cleaned dataset that you produced in last week's Workshop - either by running your cleaning script on `alcohol_acid_phys_data.csv` or by loading the cleaned dataset that you saved.

In [ ]:
# TODO:

# 1. Load the cleaned dataset

# 2. (If necessary) Figure out how to load your dataset without adding an extra index column


## 2. Getting Our Dataset Ready for RDKit

### 2.1 - Adding Molecular Identifiers (e.g., SMILES)

We have a list of compounds and a small number of observed values and descriptors. We can add a few more by calculating them using RDKit, but our dataset only contains IUPAC names. We need to obtain a more rigorous representation to use before RDKit can work its magic.

Let's use the [Chemical Identifier Resolver](https://cactus.nci.nih.gov/chemical/structure) (CIR) service is run by the CADD Group at the NCI/NIH as part of their [Cactus server](https://cactus.nci.nih.gov). It allows us to extract a range of different molecular identifiers just from the IUPAC name of a molecule.

Example code that will allow you to do this task is given below. You can also find more examples of its use in the [Molecular fingerprints notebook](https://joeforth.github.io/chem502_book/chem-data/fingerprints).

In [ ]:
# TODO:
# 
# 1. Get a list of SMILES strings for the compounds in the dataframe and add this to the dataframe as a new column. 

# 2. Save your data so you don't constantly have to run the chemical name query

# Here is a function so the process of getting the SMILES can be repeated for multiple compounds.
# It includes a sleep time (`time.sleep`) to avoid overloading the server.

def get_smiles_from_name(name):
    """Gets SMILES string from the Cactus API given a chemical name."""
    
    ROOT_URL = "https://cactus.nci.nih.gov/chemical/structure/"
    identifier = name
    representation = "smiles"

    query_url = f"{ROOT_URL}{identifier}/{representation}"

    response = requests.get(query_url)
    time.sleep(0.05)
    if response:
        return response.text
    else:
        print(f"Failed to get SMILES for {name}")
        return "not found"
        # raise Exception(f"Cactus request failed for {name}: {response.status_code}")


### 2.2 - Adding RDKit.molecule Objects to Your Dataset

Before we can calculate the descriptors, RDKit first needs `RDKit.molecule` objects to work with. Use the SMILES strings in your DataFrame to add a column containing RDKIT.molecule objects for every molecule in your datset.

You can create a separate list of molecules based on the SMILES strings in the dataframe, or you can use RDKit's [PandasTools module](https://www.rdkit.org/docs/source/rdkit.Chem.PandasTools.html) to work with them in a DataFrame.

Have a look at the [molecular fingerprints notebook](https://joeforth.github.io/chem502_book/chem-data/descriptors-similarity/) for some code to get started getting the RDKit molecules.

In [ ]:
# TODO: 

# 1. Load your saved dataset that includes chemical names.

# 2. Drop any rows where the SMILES string couldn't be found and reset the index

# 3. Add RDKit molecule objects to the dataframe - most likely using RDKit's PandasTools module

# 3. Using RDKit to Explore Molecule Properties

The aim now is to use RDKit to generate descriptors for your molecules.

## 3.1 - Calculating Molecular Descriptors

There is a [tutorial](https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html) on calculating descriptors and they are listed in the [Getting Started guide](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors).

- 1. Calculate the descriptors in "[Lipinski's Rule of 5](https://en.wikipedia.org/wiki/Lipinski%27s_rule_of_five)" for the molecules in your dataset and add them to your dataframe.

One way of doing this is to use the `getMolDescriptors` function in the [descriptors tutorial](https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html) as a starting point to calculate the new descriptors and add them to dictionary that can be read into a dataframe.
You can then use [`pd.concat`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) to combine the dataframe with your thermochemical data with the new descriptors.

- 2. Identify which compounds satisfy all conditions of Lipinski's Rule of 5.

In [ ]:
# TODO: 
# 
# 1. Calculate the 5 descriptors in "[Lipinski's Rule of 5](https://en.wikipedia.org/wiki/Lipinski%27s_rule_of_five)" for the molecules in your dataset and add them to your dataframe.
#
# 2. Identify which compounds satisfy all conditions of Lipinski's Rule of 5.

## 3.2 - Molecular Visualisation

RDKit contains a huge range of tools to visualise chemicals and chemical reactions. Let's start by getting it to visualise molecules by using the `rdkit.Chem.Draw` package.

1. The full list of commands are in the [package documentation](https://www.rdkit.org/docs/source/rdkit.Chem.Draw.html)

2. There are some examples of molecular visualisation in RDKit in the [course book page on molecular fingerprints](https://joeforth.github.io/chem502_book/chem-data/fingerprints/)

The two commands you'll need for this section are most likely [this](https://www.rdkit.org/docs/source/rdkit.Chem.Draw.html#rdkit.Chem.Draw.MolToImage) and [this](https://www.rdkit.org/docs/source/rdkit.Chem.Draw.html#rdkit.Chem.Draw.MolsMatrixToGridImage), with some nice examples in the [Getting Started Guide](https://www.rdkit.org/docs/GettingStartedInPython.html).


In [ ]:
# TODO: 
# 
# 1. Use RDKit to save a picture of 4-Decanol from your dataset - make the image resolution high enough that it's not blurry when it fills your screen
#
# 2. Use RDKit to save grid images of all the molecules in each class - make the legend / label of each molecules its SMILES string

## 3.3 - Molecular Fingerprints

Finally, let's explore the capability of RDKit to generate molecular fingerprints to describe your chemical data. 

Specifically, we're going to look at Morgan fingerprints - though there are many alternatives.

You can learn more about Morgan fingerprints from:

1. [The original paper from 1960](https://pubs.acs.org/doi/10.1021/c160017a018), which is something of a classic.

2. [A more recent review](https://pubs.acs.org/doi/full/10.1021/ci100050t), which gives more of an overview of the field.

Calculate the Morgan fingerprints of the molecules in your dataset.

You can find a summary of how to do this in the [RDKit Fingerprints tutorial](https://greglandrum.github.io/rdkit-blog/posts/2023-01-18-fingerprint-generator-tutorial.html) and [the RDKit documentation](https://www.rdkit.org/docs/GettingStartedInPython.html#explaining-bits-from-fingerprints). 

You may also find the [Morgan fingerprints section](https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints) of the RDKit documentation useful).

Finally, the [Molecular fingerprints section of the course book](https://joeforth.github.io/chem502_book/chem-data/fingerprints/) contains some useful examples of what you can do.

In [ ]:
# TODO:
#
# 1. Calculate the Morgan fingerprint of Benzoic acid. 

# Example code for generating Morgan fingerprints is given below, but there are other ways of doing it:
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

# 2. Take a look at what your Morgan fingerprint looks like - the example below displays it for a Morgan fingerprint called fp, generated using the above line of code.
bits = np.array(fp).reshape(32,64)
for row in bits:
    print(*row)



## 3.4 - Molecular Fingerprints and Pandas

Now that you've calculated a Morgan fingerprint for one molecule - calculate the Morgan fingerprint for all your molecules and add them to your DataFrame.

In [ ]:
# TODO:
# 
# 1. Calculate the Morgan fingerprints of all molecules in your data.
# 
# 2. Add your Morgan fingerprints to your DataFrame.

## 3.5 - Molecular Similarity

Finally, let's quantify how similar or dissimilar your molecules are. We'll calculate the Tanimoto similarity.

Note that Tanimoto similarity is sensitive to the type of fingerprint you've used - I found this [blog post on the matter](https://greglandrum.github.io/rdkit-blog/posts/2025-07-17-naming-similarity-metrics.html) interesting.



In [ ]:
# TODO:
#
# 1. Calculate the molecular simialarity between 1-Tetradecanol and 1-Pentadecanol
#
# 2. Calculate the molecular similarity between 1-Decanol and 4-Methyl Pentanoic Acid
#
# 3. Find the least-similar compounds in your dataset (as measured using Morgan FP and Tanimoto similarity - I found there were 5 pairs with the same value, but this will depend on your fingerprint settings)
# 
# 4. Save images of all pairs of molecules with the lowest similarity

## 4. Back to Visualisation

Over the last two workshops, you've learnt to clean chemical data using pandas, visualise chemical data using seaborn, and calculate molecular descriptors, fingerprints, and similarity using RDKit.

1. Calculate Tanimoto similarity between all possible pairs of molecules in your dataset.

2. Visualise the similarity appropriately - save the visualisation.

In [ ]:
# TODO:
# 
# 1. Calculate Tanimoto similarity between all possible pairs of molecules in your dataset.

# 2. Visualise the similarity appropriately - save the visualisation.

## Submission

Submit your answers to Workshops 1 and 2 as a single jupyter notebook that runs "out of the box".

You will be assessed on the extent to which you have answered the workshop questions, the quality of the code, and the extent to which you have explored the questions to find interesting or neat ways to solve the problems.
